In [15]:
from Utils import load_pickle, save_pickle


experiment = load_pickle('experiment','/workspace/data/output/set12/dataset_experiment/results/regional_experiment_dataset.pkl')


In [8]:
import pandas as pd
df = pd.DataFrame(experiment)

summary = df.groupby(['tipo', 'sigma']).agg(
    psnr_nlm_mean=('psnr_nlm', 'mean'),
    psnr_gnlm_mean=('psnr_gnlm', 'mean'),
    ssim_nlm_mean=('ssim_nlm', 'mean'),
    ssim_gnlm_mean=('ssim_gnlm', 'mean'),
)

print(summary)

                    psnr_nlm_mean  psnr_gnlm_mean  ssim_nlm_mean  \
tipo         sigma                                                 
mesclada     5          38.059081       37.588860       0.976274   
             50         25.100530       25.564953       0.839530   
nao_uniforme 5          38.087715       37.899614       0.973074   
             50         25.654371       25.960462       0.825853   
uniforme     5          42.523827       41.485160       0.970156   
             50         33.490322       30.804067       0.878166   

                    ssim_gnlm_mean  
tipo         sigma                  
mesclada     5            0.977670  
             50           0.737610  
nao_uniforme 5            0.976390  
             50           0.741048  
uniforme     5            0.952577  
             50           0.670070  


In [ ]:
import pickle
import pandas as pd
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

final_experiment_dataset = '/workspace/data/output/set12/dataset_experiment/results/final_experiment_dataset.pkl'

# ============================================
# LOAD DATA
# ============================================

with open("/workspace/data/output/set12/dataset_experiment/results/regional_experiment_dataset.pkl", "rb") as f:
    dataset = pickle.load(f)

save_path = save_pickle(dataset, output_dir, filename)



df_psnr = pd.DataFrame(results_psnr)
df_psnr["delta_psnr"] = df_psnr["psnr_gnlm"] - df_psnr["psnr_nlm"]

# ============================================
# EXTRACT STRUCTURAL METRICS
# ============================================

rows = []

for tipo in dataset:
    for entry in dataset[tipo]:
        idx = entry["index"]

        for sigma in [5, 50]:

            spectral_ratio = entry["patch_metrics"][sigma]["spectral_ratio"]

            rows.append({
                "tipo": tipo,
                "index": idx,
                "sigma": sigma,
                "spectral_ratio": spectral_ratio
            })

df_struct = pd.DataFrame(rows)

# ============================================
# MERGE WITH PSNR RESULTS
# ============================================

df = df_psnr.merge(
    df_struct,
    on=["tipo", "index", "sigma"]
)

# ============================================
# CORRELATION
# ============================================

print("\n==============================")
print("CORRELATION ANALYSIS")
print("==============================")

# Global
corr_global, p_global = pearsonr(df["spectral_ratio"], df["delta_psnr"])

print("\nGLOBAL:")
print(f"r = {corr_global:.4f}")
print(f"p-value = {p_global:.6f}")

# By sigma
for sigma in [5, 50]:
    sub = df[df["sigma"] == sigma]
    corr, pval = pearsonr(sub["spectral_ratio"], sub["delta_psnr"])

    print(f"\nSigma {sigma}:")
    print(f"r = {corr:.4f}")
    print(f"p-value = {pval:.6f}")

# ============================================
# SCATTER PLOT
# ============================================

plt.figure(figsize=(6,5))
plt.scatter(df["spectral_ratio"], df["delta_psnr"], alpha=0.7)
plt.axhline(0, color="red", linestyle="--")
plt.xlabel("Spectral Ratio")
plt.ylabel("ΔPSNR (GEO - NLM)")
plt.title("SpectralRatio vs ΔPSNR")
plt.tight_layout()
plt.show()

    

EOFError: Ran out of input